In [6]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.feature_selection import mutual_info_regression



def main():
    #全時系列データの呼び出し
    stock= 'sandp500_2'
    path = '../../stock_data/time_series_data/{}'.format(stock)
    stock_datas = os.listdir(path)
#     print(stock_datas)
    
    mi_score = pd.DataFrame()
    for i in stock_datas:
        stock_data = pd.read_csv('../../stock_data/time_series_data/{}/'.format(stock) + i)
        df = stock_data.copy()
        df = df.set_index('Date')
        #範囲を多めに取って計算量を減らす
        df = df['2010-01-01':'2021-03-01']
 
        try:
            feature1(df)
            
            df = df['2015-01-01':'2020-01-01']
    #         #データセットの用意
            X = df.drop(['Open','High','Low','Close','Adj Close','Volume','validation'],axis='columns') 
            y = df['validation']
            print(i)
            print(X)
            print(y)
     
        # All discrete features should now have integer dtypes (double-check this before using MI!)
            discrete_features = X.dtypes == int
        #相互情報量を調査
            mi_scores = make_mi_scores(X, y, discrete_features)
            print(mi_scores)
            mi_score = pd.concat([mi_score, mi_scores], axis=1)
        except:
            continue
            
    #スコアの平均を求める
    score = mi_score.mean(axis='columns')
    print(score)
    scores_sort = score.sort_values(ascending=True)
    print('----------------------------')
    print('ポイント高い順')
    print(scores_sort)
    
    #グラフで描写する
    plt.figure(dpi=100, figsize=(8, 16))
    plot_mi_scores(score)
    plt.show()

    
    
    
# 加重移動平均を求める関数
def wma(milktea):
    
    weight = np.arange(len(milktea)) + 1
    wma = np.sum(weight * milktea) / weight.sum()
    return wma

def TrueRange(c, h, l, o, yc):
    x = h-l
    y = abs(h-yc)
    z = abs(l-yc)
    if y <= x >= z:
        TR = x
    elif x <= y >= z:
        TR = y
    elif x <= z >= y:
        TR = z
    return TR

def PDM(o, h, l, c, yo, yh, yl, yc):
    moveUp = h - yh
    moveDown = yl - l
    if 0 < moveUp > moveDown:
        PDM = moveUp
    else:
        PDM = 0
        
    return PDM

def NDM(o, h, l, c, yo, yh, yl, yc):
    moveDown = yl - l
    moveUp = h - yh
    if 0 < moveDown > moveUp:
        NDM = moveDown
    else:
        NDM = 0
    
    return NDM

def feature1(df):
    # simple moving average
    for i in [30,60,90,120,150,180,210,240,270,300]:
        df['sma'+str(i)] = df['Adj Close'].rolling(i).mean()
        df['sma'+str(i)] = df['sma'+str(i)] / df['sma'+str(i)].shift(1)
        df['sma'+str(i)] = Zero_One_Scale(df['sma'+str(i)])
    for i in range(15,150,30):
        for k in range(30,270,60):
            df['ratio_sma'+str(k)] = df['Adj Close'].rolling(k).mean()
            df['ratio_sma'+str(i)] = df['Adj Close'].rolling(i).mean()
            df['ratio_sma'+str(i)+'_'+str(k)] = df['ratio_sma'+str(i)] / df['ratio_sma'+str(k)]
            df['ratio_sma'+str(i)+'_'+str(k)] = Zero_One_Scale(df['ratio_sma'+str(i)])

        
    for term in range(5,50,5):
        df['SMA'+str(term)] = df['Adj Close'].rolling(term).mean()
        df['STD'+str(term)] = df['Adj Close'].rolling(term).std()
        df['Standard_deviation_normalization'+str(term)] = 100 * 2 * df['STD'+str(term)] / df['SMA'+str(term)]
        df['Standard_deviation_normalization'+str(term)] = Zero_One_Scale(df['Standard_deviation_normalization'+str(term)])
        
    for i in range(41,201,40):
        df['Highest'+str(i)] = df['Adj Close'].rolling(window=81).max()
        df['Highest'+str(i)] = df['Highest'+str(i)].shift()
        for m in range(30,90,30):
            df['Highest'+str(i)+','+str(m)+'days_ago'] = df['Adj Close'] / df['Highest'+str(i)].shift(m)
            df['Highest'+str(i)+','+str(m)+'days_ago'] = Zero_One_Scale(df['Highest'+str(i)+','+str(m)+'days_ago'])
        
        #今日の終値が過去何日間の高音に対してどの程度あるか
        df['Highest'+str(i)] = df['Adj Close'] / df['Highest'+str(i)]
        df['Highest'+str(i)] = Zero_One_Scale(df['Highest'+str(i)])
       
    for i in [5,10,14,20,50,75,100,150,200,250]:
        df["wma"+str(i)]=df["Adj Close"].rolling(i).apply(wma, raw=True).round(1)
        df["ema"+str(i)]=df["Adj Close"].ewm(span=i).mean()
        df["wma"+str(i)] = df["wma"+str(i)] / df["wma"+str(i)].shift(1)
        df["ema"+str(i)] = df["ema"+str(i)] / df["ema"+str(i)].shift(1)
        df["wma"+str(i)] = Zero_One_Scale(df["wma"+str(i)])
        df["ema"+str(i)] = Zero_One_Scale(df["ema"+str(i)])
    
    
    df["TrueRange"] = np.nan
    df["PDM"] = np.nan
    df["NDM"] = np.nan
    
    for i in range(0,len(df)):
        df["TrueRange"][i] = TrueRange(df["Adj Close"][i],df["High"][i],df["Low"][i],df["Open"][i],df["Adj Close"][i-1])
        df["PDM"][i] = PDM(df["Open"][i],df["High"][i],df["Low"][i],df["Adj Close"][i],df["Open"][i-1],df["High"][i-1],df["Low"][i-1],df["Adj Close"][i-1])
        df["NDM"][i] = NDM(df["Open"][i],df["High"][i],df["Low"][i],df["Adj Close"][i],df["Open"][i-1],df["High"][i-1],df["Low"][i-1],df["Adj Close"][i-1])
    
    df['PDI'] = df["PDM"].rolling(14).sum()/df["TrueRange"].rolling(14).sum() * 100
    df['NDI'] = df["NDM"].rolling(14).sum()/df["TrueRange"].rolling(14).sum() * 100

    
    df['DX'] = (df['PDI']-df['NDI']).abs()/(df['PDI']+df['NDI']) * 100
    df['DX'] = df['DX'].fillna(0)
    
    df['ADX'] = df['DX'].rolling(14).mean()
    df['ADXR'] = df['ADX'].rolling(14).mean()
    
    df['ADX'] = Zero_One_Scale(df['ADX'])
    df['ADXR'] = Zero_One_Scale(df['ADXR'])
    
#     calucurate A/D
    AD = []
    AD.append(0)
    for i in range(1, len(df)):
        AD_component = ((df["Adj Close"][i] - df["Low"][i]) - (df["High"][i] - df["Adj Close"][i])) * df['Volume'][i] / (df["High"][i] - df["Low"][i]) + AD[-1]
        AD.append(AD_component)

    df['A/D'] = AD
    df['A/D_EMA'] = df['A/D'].ewm(com=20).mean()   
    df['A/D_ratio'] = df['A/D'] / df['A/D_EMA']
    df['ADOSC'] = df['A/D'].ewm(com=3).mean() / df['A/D'].ewm(com=10).mean()   
    df['A/D_EMA_ratio'] = df['A/D_EMA'] / df['A/D_EMA'].shift(1)
                
    df['A/D'] = Zero_One_Scale(df['A/D'])
    df['A/D_EMA'] = Zero_One_Scale(df['A/D_EMA'])
    df['A/D_ratio'] = Zero_One_Scale(df['A/D_ratio'])
    df['A/D_EMA_ratio'] = Zero_One_Scale(df['A/D_ratio'])
    df['ADOSC'] = Zero_One_Scale(df['ADOSC'])
    
    
    
    OBV = []
    OBV.append(0)
    for i in range(1, len(df.Close)):
        if df.Close[i] > df.Close[i-1]: #If the closing price is above the prior close price 
              OBV.append(OBV[-1] + df.Volume[i]) #then: Current OBV = Previous OBV + Current Volume
        elif df.Close[i] < df.Close[i-1]:
              OBV.append( OBV[-1] - df.Volume[i])
        else:
              OBV.append(OBV[-1])
                
    #Store the OBV and OBV EMA into new columns
    df['OBV'] = OBV
    df['OBV_EMA'] = df['OBV'].ewm(com=20).mean()   
    df['OBV_ratio'] = df['OBV'] / df['OBV_EMA']
    
    df['OBV_EMA_ratio'] =  df['OBV_EMA'] / df['OBV_EMA'].shift(1)
                
    df['OBV'] = Zero_One_Scale(df['OBV'])
    df['OBV_EMA'] = Zero_One_Scale(df['OBV_EMA'])
    df['OBV_ratio'] = Zero_One_Scale(df['OBV_ratio'])
    df['OBV_EMA_ratio'] = Zero_One_Scale(df['OBV_EMA_ratio'])
    validation(df)
        
        
    #calucurate aroon
    for periods in [14,20]:
        df['aroon_up'+str(periods)] = df['High'].rolling(periods+1).apply(lambda x: x.argmax(), raw=True) / periods * 100
        df['aroon_down'+str(periods)] = df['Low'].rolling(periods+1).apply(lambda x: x.argmin(), raw=True) / periods * 100
        df['aroon_ratio'+str(periods)] = df['aroon_up'+str(periods)] / df['aroon_down'+str(periods)]
        df['AROONOSC'+str(periods)] = df['aroon_up'+str(periods)] - df['aroon_down'+str(periods)]

        df['aroon_up'+str(periods)] = Zero_One_Scale(df['aroon_up'+str(periods)])
        df['aroon_down'+str(periods)] = Zero_One_Scale(df['aroon_down'+str(periods)])
        df['aroon_ratio'+str(periods)] = Zero_One_Scale(df['aroon_up'+str(periods)])
        df['AROONOSC'+str(periods)] = Zero_One_Scale(df['AROONOSC'+str(periods)])

    #calucurate bop
    df['df_bop'] = (df['Close'] - df['Open'])/(df['High'] - df['Low'])
    df['ema_bop'] = df['df_bop'].ewm(span=14).mean()
    
    
    
    
    
    
    # feature calculation
    # basic information
    df['price-change'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df['price-change-percentage'] = df['Adj Close'] / df['Adj Close'].shift(1)
    df['volume'] = Zero_One_Scale(df['Volume'])
    df['amount'] = df['Adj Close'] * df['Volume']
    df['amount'] = Zero_One_Scale(df['amount'])
    # simple moving average
    df['sma10'] = df['Adj Close'].rolling(30).mean()
    df['sma10-FP'] = (df['sma10'] - df['sma10'].shift(1)) / df['sma10'].shift(1)
    df['sma10-FP'] = One_One_Scale(df['sma10-FP'])
    df['sma10'] = Zero_One_Scale(df['sma10'])
    # Moving Average Convergence Divergence
    df['macd'] = df['Adj Close'].rolling(12).mean() - df['Adj Close'].rolling(26).mean()
    df['macd-SG'] = df['macd'].rolling(9).mean()
    df['macd-histogram'] = df['macd'] - df['macd-SG']
    df['macd-histogram'] = np.where(df['macd-histogram'] > 0, 1, -1)
    df['macd-SG'] = np.where(df['macd-SG'] > 0, 1, -1)
    df['macd'] = np.where(df['macd'] > 0, 1, -1)
    # Commodity Channel Index in 24 days
    df['typical-price'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['sma-cci'] = df['typical-price'].rolling(24).mean()
    df['mean-deviation'] = np.abs(df['typical-price'] - df['sma-cci'])
    df['mean-deviation'] = df['mean-deviation'].rolling(24).mean()
    df['cci'] = (df['typical-price'] - df['sma-cci']) / (0.015 * df['mean-deviation'])
    df['cci-SG'] = np.where(df['cci'] > 0, 1, -1)
    # MTM 10
    df['mtm10'] = df['Adj Close'] - df['Adj Close'].shift(10)
    df['mtm10'] = np.where(df['mtm10'] > 0, 1, -1)
    # Rate of Change in 10 days
    df['roc'] = (df['Adj Close'] - df['Adj Close'].shift(10)) / df['Adj Close'].shift(10)
    df['roc-SG'] = np.where(df['roc'] > 0, 1, -1)
    df['roc-FP'] = (df['roc'] - df['roc'].shift(1))
    df['roc-FP'] = One_One_Scale(df['roc-FP'])

    # ADOSC
    df['adosc'] = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
    df['adosc'] = df['adosc'].cumsum()
    df['adosc-ema3'] = df['adosc'].ewm(span=3, adjust=False).mean()
    df['adosc-ema10'] = df['adosc'].ewm(span=10, adjust=False).mean()
    df['adosc-SG'] = np.where((df['adosc-ema3'] - df['adosc-ema10']) > 0, 1, -1)
    df['adosc'] = Zero_One_Scale(df['adosc'])
    # AR 26
    hp_op = (df['High'] - df['Open']).rolling(26).sum()
    op_lp = (df['Open'] - df['Low']).rolling(26).sum()
    df['ar26'] = hp_op / op_lp
    df['ar26'] = Zero_One_Scale(df['ar26'])
    # BR 26
    hp_cp = (df['High'] - df['Close']).rolling(26).sum()
    cp_lp = (df['Close'] - df['Low']).rolling(26).sum()
    df['br26'] = hp_cp / cp_lp
    df['br26'] = Zero_One_Scale(df['br26'])
    # VR 26
    
    # BIAS 20
    sma20 = df['Adj Close'].rolling(20).mean()
    df['bias20'] = (df['Adj Close'] - sma20) / sma20
    df['bias20'] = np.where(df['bias20'] > 0, 1, -1)
    
    return df

def validation(df):
    k=10
    df["validation"] = np.nan
    for i in range(0,len(df)-k):
        df["validation"][i] = df['Adj Close'][i+k]/df['Adj Close'][i]
        
    df["validation"] = Zero_One_Scale(df["validation"])

    return df
 

        
    
def Zero_One_Scale(df):
    df_scaled = (df - df.min()) / (df.max() - df.min())
    return df_scaled

def One_One_Scale(df):
    df_scaled = 2 * (df - df.min()) / (df.max() - df.min()) - 1
    return df_scaled

def Normalize(df):
    df_normalized = (df - df.mean(axis=0)) / df.std(axis=0)


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

    
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


        
        
main()

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_

CSCO.csv
               sma30     sma60     sma90    sma120    sma150    sma180  \
Date                                                                     
2015-01-02  0.642435  0.626004  0.721344  0.634369  0.717427  0.734007   
2015-01-05  0.607406  0.591711  0.686240  0.593845  0.686068  0.701946   
2015-01-06  0.592611  0.618596  0.686075  0.577382  0.689852  0.693188   
2015-01-07  0.604251  0.683127  0.696520  0.608272  0.708548  0.698817   
2015-01-08  0.608159  0.718979  0.700137  0.604773  0.709050  0.708664   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.552888  0.397311  0.581850  0.276801  0.403201  0.369277   
2019-12-26  0.545139  0.454350  0.540121  0.289311  0.437404  0.357180   
2019-12-27  0.538544  0.490381  0.554016  0.282856  0.431597  0.359988   
2019-12-30  0.669169  0.456356  0.525017  0.256537  0.437135  0.341452   
2019-12-31  0.676316  0.453383  0.552354  0.261593  0.461702  0.343526   

              sma210    sma2

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_

UAL.csv
               sma30     sma60     sma90    sma120    sma150    sma180  \
Date                                                                     
2015-01-02  0.653141  0.882131  0.864587  0.865533  0.855576  0.822988   
2015-01-05  0.657182  0.892494  0.864919  0.860205  0.831092  0.824310   
2015-01-06  0.653659  0.870013  0.849054  0.847777  0.808127  0.807138   
2015-01-07  0.668714  0.886335  0.856734  0.870648  0.822752  0.796727   
2015-01-08  0.672314  0.917907  0.873672  0.871875  0.831535  0.804232   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.567808  0.723546  0.727062  0.654662  0.685973  0.640136   
2019-12-26  0.572635  0.726840  0.722084  0.654826  0.691763  0.639305   
2019-12-27  0.569488  0.742628  0.715999  0.656579  0.687594  0.628396   
2019-12-30  0.566752  0.742657  0.706157  0.654056  0.690150  0.627622   
2019-12-31  0.567455  0.738334  0.705328  0.654932  0.694314  0.633275   

              sma210    sma24

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_

...

UAA.csv
               sma30     sma60     sma90    sma120    sma150    sma180  \
Date                                                                     
2015-01-02  0.672035  0.645926  0.569726  0.593869  0.628066  0.602535   
2015-01-05  0.641008  0.616095  0.553407  0.586134  0.610404  0.595413   
2015-01-06  0.629236  0.626322  0.550557  0.585697  0.604802  0.587305   
2015-01-07  0.668843  0.664869  0.574389  0.606602  0.603287  0.598710   
2015-01-08  0.680474  0.701442  0.600990  0.607296  0.605302  0.607442   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.872781  0.694931  0.693884  0.422259  0.442307  0.520977   
2019-12-26  0.890037  0.707343  0.694957  0.410987  0.465041  0.510122   
2019-12-27  0.876445  0.711601  0.690804  0.399136  0.454916  0.501307   
2019-12-30  0.872670  0.704730  0.679317  0.398858  0.451121  0.487068   
2019-12-31  0.857049  0.714356  0.681960  0.403247  0.475794  0.493169   

              sma210    sma24

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_

BLK.csv
               sma30     sma60     sma90    sma120    sma150    sma180  \
Date                                                                     
2015-01-02  0.608480  0.577762  0.552886  0.652638  0.627101  0.505879   
2015-01-05  0.582027  0.525140  0.511222  0.621251  0.598173  0.484334   
2015-01-06  0.559355  0.539552  0.502977  0.599246  0.576012  0.475830   
2015-01-07  0.555608  0.585842  0.535687  0.646006  0.590182  0.489706   
2015-01-08  0.570093  0.616377  0.543476  0.649265  0.598341  0.503114   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.612032  0.584399  0.672630  0.598788  0.609783  0.499245   
2019-12-26  0.620974  0.622811  0.666343  0.621671  0.627225  0.500582   
2019-12-27  0.623355  0.654812  0.682705  0.617879  0.622899  0.493316   
2019-12-30  0.620304  0.650623  0.670699  0.608759  0.631037  0.472388   
2019-12-31  0.609135  0.635328  0.679329  0.602611  0.637283  0.481044   

              sma210    sma24

/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/stock/lib/python3.7/site-packages/ipykernel_

PPL.csv
               sma30     sma60     sma90    sma120    sma150    sma180  \
Date                                                                     
2015-01-02  0.705295  0.706156  0.679339  0.654008  0.560477  0.641704   
2015-01-05  0.684321  0.653821  0.661347  0.615053  0.539562  0.620513   
2015-01-06  0.688366  0.672746  0.646761  0.609972  0.531291  0.627247   
2015-01-07  0.685371  0.675435  0.647228  0.628501  0.542317  0.633093   
2015-01-08  0.692740  0.675302  0.634859  0.603317  0.535012  0.621732   
...              ...       ...       ...       ...       ...       ...   
2019-12-24  0.752405  0.762435  0.863839  0.739849  0.752743  0.690160   
2019-12-26  0.745492  0.765248  0.857723  0.734843  0.735945  0.696474   
2019-12-27  0.742165  0.790596  0.852959  0.748217  0.717319  0.686928   
2019-12-30  0.753969  0.795854  0.849618  0.768341  0.732230  0.697112   
2019-12-31  0.751060  0.790072  0.855285  0.772290  0.770231  0.704641   

              sma210    sma24